In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

import pymongo


client = pymongo.MongoClient('mongodb://donkey:27117/')
db = client.sphere_kaggle

In [3]:
client = pymongo.MongoClient('mongodb://donkey:27117/')
db = client.sphere_kaggle

In [4]:
db.collection_names()

[u'debug',
 u'errors',
 u'results',
 u'system.indexes',
 u'test',
 u'test_hashtag_data',
 u'test_text_data',
 u'test_tweets',
 u'train_hashtag_data',
 u'train_text_data',
 u'train_tweets',
 u'tweets',
 u'unrecognized_locations',
 u'users']

#### Training users

In [5]:
train_users_ids = db.users.distinct("id", {"label": {"$exists": True}})
len(train_users_ids)

2981

In [6]:
train_users_ids_with_tweets = db.train_tweets.distinct("user.id")
len(train_users_ids_with_tweets)

2684

In [7]:
train_users_ids_without_tweets = set(train_users_ids) - set(train_users_ids_with_tweets)
len(train_users_ids_without_tweets)

298

#### Test users

In [8]:
test_users_ids = db.users.distinct("id", {"label": {"$exists": False}})
len(set(test_users_ids))

1993

In [9]:
test_users_ids_with_tweets = db.test_tweets.distinct("user.id")
len(set(test_users_ids_with_tweets))

1990

In [10]:
test_users_ids_without_tweets = set(test_users_ids) - set(test_users_ids_with_tweets)
len(set(test_users_ids_without_tweets))

30

In [11]:
from lib.downloaders import TweetsDownloader


In [21]:
twtDown = TweetsDownloader(tweets_collection=db.test_tweets,
                 users_collection=db.users,
                 errors_collection=db.errors)

In [22]:
twtDown.download(test_users_ids_without_tweets)

DEBUG:twitter_daemon.285438978:start loading user
ERROR:log:Twitter error response: status code = 401
DEBUG:twitter_daemon.473204360:start loading user
ERROR:log:Twitter error response: status code = 401
DEBUG:twitter_daemon.552233230:start loading user
ERROR:log:Twitter error response: status code = 401
DEBUG:twitter_daemon.249136400:start loading user
ERROR:log:Twitter error response: status code = 401
DEBUG:twitter_daemon.2586452886:start loading user
ERROR:log:Twitter error response: status code = 401
DEBUG:twitter_daemon.202331160:start loading user
ERROR:log:Twitter error response: status code = 401
DEBUG:twitter_daemon.286246563:start loading user
ERROR:log:Twitter error response: status code = 401
DEBUG:twitter_daemon.18092837:start loading user
ERROR:log:Twitter error response: status code = 401
DEBUG:twitter_daemon.166904878:start loading user
ERROR:log:Twitter error response: status code = 401
DEBUG:twitter_daemon.1667482549:start loading user
ERROR:log:Twitter error respons

In [ ]:
db.train_text_data.count()

In [ ]:
train_tweets_cursor = db.train_tweets.aggregate(
    [{"$group": {
        "_id": "$user.id",
        "count": { "$sum": 1 }
    }}]
)

In [ ]:
train_tweets = list(train_tweets_cursor)

In [ ]:
users_ids_in_train_tweets = db.train_tweets.distinct("user.id")

In [ ]:
set(train_users_ids) - set(users_ids_in_train_tweets)

In [16]:
tf1 = TfidfVectorizer()
tf1.fit_transform(["snow sheldon fog", "fog dog"]).toarray()

array([[ 0.        ,  0.44943642,  0.6316672 ,  0.6316672 ],
       [ 0.81480247,  0.57973867,  0.        ,  0.        ]])

In [ ]:
tf2 = TfidfVectorizer()
tf2.fit(["snow sheldon", "saw"]);

In [ ]:
print tf1.transform(["dog fog saw"]).toarray()
print tf2.transform(["dog fog saw"]).toarray()

In [ ]:
print tf1.get_feature_names()#	Array mapping from feature integer indices to feature name
print tf2.get_feature_names()#	Array mapping from feature integer indices to feature name

In [58]:
td1 = TfidfVectorizer(vocabulary=["word1", "word2", "word3", "word4"], sublinear_tf=False, use_idf=False, norm=None)

In [59]:
td1.fit_transform([
        "word1 word3 word3 word2 word2 word3", # 0
        "word1 word1 word3 word2 word2 word2", # 0 
        "word1 word3 word3 word2 word2 word2", # 0
        "word1 word1 word1 word2 word3 word2", # 0
        "word4 word4 word1 word2 word2 word4", # 1
        "word4 word4 word1 word2 word3 word2", # 1
        "word4 word4 word1 word2 word4 word2", # 1
        "word4 word4 word1 word3 word2 word2", # 1
    ]).toarray()

array([[ 1.,  2.,  3.,  0.],
       [ 2.,  3.,  1.,  0.],
       [ 1.,  3.,  2.,  0.],
       [ 3.,  2.,  1.,  0.],
       [ 1.,  2.,  0.,  3.],
       [ 1.,  2.,  1.,  2.],
       [ 1.,  2.,  0.,  3.],
       [ 1.,  2.,  1.,  2.]])

In [37]:
td1.get_feature_names()

['word1', 'word2', 'word3', 'word4']

In [ ]:
d1 = ["word1 is whery greate"]
d1 = []